In [1]:
import fastai
fastai.__version__

'1.0.54'

!pip install -U fastai==1.0.60

In [3]:
!sh download_fastai_sentiment_model.sh

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100   408    0   408    0     0    758      0 --:--:-- --:--:-- --:--:--   758
  0     0    0     0    0     0      0      0 --:--:--  0:00:01 --:--:--     0
  0     0    0     0    0     0      0      0 --:--:--  0:00:01 --:--:--     0
100  290M    0  290M    0     0  7945k      0 --:--:--  0:00:37 --:--:-- 9354k


In [4]:
!du -h sentiment_model.tgz

291M	sentiment_model.tgz


In [7]:
!tar xzvf sentiment_model.tgz

storage/sentiment/
storage/sentiment/models/
storage/sentiment/models/third.pth
storage/sentiment/data_clas.pkl


In [5]:
!mv storage model

mv: cannot move 'storage' to 'model/storage': Operation not permitted


In [2]:
from pathlib import Path
from fastai.text import load_data, text_classifier_learner, AWD_LSTM
bs=48
path=Path('./model/sentiment')
data_clas = load_data(path, 'data_clas.pkl', bs=bs)
learn = text_classifier_learner(data_clas, AWD_LSTM, drop_mult=0.5)


/home/mando/miniconda3/lib/python3.8/site-packages/torch/cuda/__init__.py:52: UserWarning: CUDA initialization: Found no NVIDIA driver on your system. Please check that you have an NVIDIA GPU and installed a driver from http://www.nvidia.com/Download/index.aspx (Triggered internally at  /pytorch/c10/cuda/CUDAFunctions.cpp:100.)
  return torch._C._cuda_getDeviceCount() > 0
/home/mando/miniconda3/lib/python3.8/site-packages/torch/serialization.py:658: SourceChangeWarning: source code of class 'torch.nn.modules.loss.CrossEntropyLoss' has changed. you can retrieve the original source code by accessing the object's source attribute or set `torch.nn.Module.dump_patches = True` and use the patch tool to revert the changes.
  warnings.warn(msg, SourceChangeWarning)
/home/mando/miniconda3/lib/python3.8/site-packages/numpy/core/_asarray.py:102: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengt

In [3]:
# pytorch model under the hood
learn.load('third')
learn.model.eval()

SequentialRNN(
  (0): MultiBatchEncoder(
    (module): AWD_LSTM(
      (encoder): Embedding(30832, 400, padding_idx=1)
      (encoder_dp): EmbeddingDropout(
        (emb): Embedding(30832, 400, padding_idx=1)
      )
      (rnns): ModuleList(
        (0): WeightDropout(
          (module): LSTM(400, 1152, batch_first=True)
        )
        (1): WeightDropout(
          (module): LSTM(1152, 1152, batch_first=True)
        )
        (2): WeightDropout(
          (module): LSTM(1152, 400, batch_first=True)
        )
      )
      (input_dp): RNNDropout()
      (hidden_dps): ModuleList(
        (0): RNNDropout()
        (1): RNNDropout()
        (2): RNNDropout()
      )
    )
  )
  (1): PoolingLinearClassifier(
    (layers): Sequential(
      (0): BatchNorm1d(1200, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (1): Dropout(p=0.2, inplace=False)
      (2): Linear(in_features=1200, out_features=50, bias=True)
      (3): ReLU(inplace=True)
      (4): BatchNorm1d(50, 

In [4]:
example_message = "I don't know why I keep focusing on negative things. What a bummer!"

In [5]:
learn.predict(example_message)

/home/mando/miniconda3/lib/python3.8/site-packages/fastai/text/data.py:338: UserWarning: This overload of nonzero is deprecated:
	nonzero()
Consider using one of the following signatures instead:
	nonzero(*, bool as_tuple) (Triggered internally at  /pytorch/torch/csrc/utils/python_arg_parser.cpp:882.)
  idx_min = (t != self.pad_idx).nonzero().min()


(Category neg, tensor(0), tensor([0.9182, 0.0818]))

In [6]:
learn.predict('I love this workshop')

(Category pos, tensor(1), tensor([0.0385, 0.9615]))

In [7]:
from typing import Tuple
def predict_sentiment(sentence:str) -> Tuple[str, float]:
    categorical, class_id, scores = learn.predict(sentence)
    score = round(scores[class_id].item(), 4)
    return "negative" if class_id == 0 else "positive", score

In [8]:
predict_sentiment(example_message)

('negative', 0.9182)

# Add step to the pipeline

In [17]:
import redis
r = redis.Redis()
sub = r.pubsub(ignore_subscribe_messages=True)
sub.subscribe(['NER'])

In [18]:
import mq
from tqdm import tqdm

In [22]:
for message in tqdm(sub.listen()):
    data = mq.read_message_data(message)
    print(data)
    entities = predict_sentiment(data['sentence'])
    #print(sentiment, score)
    for e in entities:
        output = dict(
        entity=e,
        sentiment=sentiment,
        score=score
        )
    r.publish('ready', mq.serialize_message_data(output))
    

0it [00:00, ?it/s]

{'sentence': 'r/ContrarianPod has all relevant info', 'entities': ['ContrarianPod']}


1it [00:00,  2.30it/s]

{'sentence': ' I found GME before the first squeeze and deeply regretted not getting on it', 'entities': ['GME', 'first']}


2it [00:00,  2.27it/s]

...

{'sentence': ' The only problem is the 6 flat fee for otc', 'entities': ['6']}


67it [02:12,  1.12it/s]

{'sentence': ' The only problem is the 6 flat fee for otc', 'entities': ['6']}


68it [02:24,  2.13s/it]


KeyboardInterrupt: 

### Fallback always-happy sentiment in case the WiFi fails

In [20]:
import random

In [21]:
def decide_sentiment(sentence:str) -> Tuple[str, float]:
    return 'positive', round(random.uniform(.5, 1), 3)